# <span style="color:Green"> Asteroid 2024 YR4 Tracking  </span>

**SBDB**- Small Body Database
**Sentry** -Earth Impact Monitoring System

In [44]:
import requests
import pandas as pd


In [50]:
#Pulling data from SDBD
sbdb_url = "https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=2024%20YR4"
response = requests.get(sbdb_url)
data = response.json()
sbdb_df = pd.DataFrame(data["orbit"]["elements"])
sbdb_df.to_csv("2024_YR4_SBDB_Data.csv", mode="a", header=not pd.io.common.file_exists("2024_YR4_SBDB_Data.csv"))

In [54]:
sbdb_df.head()

,value,name,units,title,label,sigma
0,0.662,e,None,eccentricity,e,5.1e-06
1,2.52,a,au,semi-major axis,a,3.5e-05
2,0.851,q,au,perihelion distance,q,7.4e-07
3,3.41,i,deg,inclination; angle with respect to x-y eclipti...,i,1.9e-05
4,271,om,deg,longitude of the ascending node,node,9.7e-06


In [52]:
print(sbdb_df[["name","value","sigma"]])

   name        value    sigma
0     e        0.662  5.1e-06
1     a         2.52  3.5e-05
2     q        0.851  7.4e-07
3     i         3.41  1.9e-05
4    om          271  9.7e-06
5     w          134  2.2e-05
6    ma         40.4  0.00087
7    tp  2460636.918  5.6e-05
8   per         1460    0.031
9     n        0.247  5.2e-06
10   ad         4.18  5.9e-05


In [20]:
# Pullig data from Sentry data base
sentry_url = "https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=2024%20YR4"
response = requests.get(sentry_url)
data = response.json()
sentry_df = pd.DataFrame([data["orbit"]["elements"]])
sentry_df.to_csv("2024_YR4_SBDB_Data.csv", mode="a", header=not pd.io.common.file_exists("2024_YR4_data.csv"))